<img src="rocky-question1b.jpg">

# Solution

## Step 2: To test the regex

In [1]:
import re

# To match the brackets, which is "(...)"
pattern2 = r"\((\w.+?)\)"  # <-- Use the question mark to match as few characters as possible (non-greedy).
pattern3 = r"\((\w.+)\)"   # <-- This performs 'greedy' match.

In [2]:
# Test 1
sentence = "A and (B or C) and (D or E) and (F or G)"
m = re.finditer(pattern2, sentence)
[x.span() for x in m]

[(6, 14), (19, 27), (32, 40)]

In [3]:
# Test 2
word = 'or'
sentence = "A and (B or C) and (D or E) and (F or G)"
m = re.finditer(word, sentence)
[x.span() for x in m]

[(9, 11), (22, 24), (35, 37)]

In [4]:
# Test 3
word = 'and'
sentence = "A and (B or C) and (D or E) and (F or G)"
m = re.finditer(word, sentence)
[x.span() for x in m]

[(2, 5), (15, 18), (28, 31)]

In [5]:
# Test 4
word = 'xor'
sentence = "A and (B or C) and (D or E) and (F or G)"
m = re.finditer(word, sentence)
[x.span() for x in m]  # <-- This gives an empty list, when the expression is not matched.

[]

In [6]:
# function

def is_inside_bracket(position:tuple, iBrackets:list) -> bool:
    for b in iBrackets:
        if b[0]< position[0] and b[1] > position[1]:
            return True
    return False

In [7]:
# Tests
assert is_inside_bracket((8, 11), [(5, 14), (19, 27), (32, 40)]) is True
assert is_inside_bracket((16, 18), [(5, 14), (19, 27), (32, 40)]) is False
assert is_inside_bracket((29, 31), [(5, 14), (19, 27), (32, 40)]) is False
assert is_inside_bracket((16, 18), []) is False

In [8]:
# This get the position of the word, for which the word MUST be outside the brackets.
# Otherwise return None.

def get_position(word:str, sentence:str) -> tuple:
    iX = [x.span() for x in re.finditer(word, sentence)]
    iBrackets = [x.span() for x in re.finditer(pattern2, sentence)]
    for y in iX:
        if not is_inside_bracket(y, iBrackets):
            return y  # <type tuple>
    return None

In [9]:
# Tests
assert get_position('or', 'A or (B and C) and (D or E) and (F or G)') == (2, 4)
assert get_position('or', 'A and (B or C) and (D or E) and (F or G)') is None  # <-- There is no 'or' outside the brackets.
assert get_position('or', 'A and (B or C) and (D or E) and (F or G) or Z') == (41, 43)
assert get_position('and', 'A or (B and C) and (D or E) and (F or G)') == (15, 18)
assert get_position('and', 'A or (B and C) or (D and E) or (F and G)') == None # <-- There is no 'and' outside the brackets.

## This must be correct first: (below)

In [10]:
# def get_position(word:str, sentence:str) -> tuple:
    # iX = [x.span() for x in re.finditer(word, sentence)]
    # iBrackets = [x.span() for x in re.finditer(pattern3, sentence)]
    # for y in iX:
        # if not is_inside_bracket(y, iBrackets):
            # return y  # <type tuple>
    # return None
# 
# assert get_position('and', '(((B or C) and D) and E)') is None   
# assert get_position('and', '((B or C) and D) and E') == (17, 20)
# assert get_position('or', '((B or C) and D) and E') is None

In [11]:
# function

def split_sentence(sentence:str) -> tuple:
    pOR = get_position(' or ', sentence)    # <type tuple>
    pAND = get_position(' and ', sentence)  # <type tuple>

    if pOR and (pAND is None or pAND[0] > pOR[0]):
        return sentence[:pOR[0]], 'OR', sentence[pOR[1]:]
    
    elif pAND and (pOR is None or pOR[0] > pAND[0]):
        return sentence[:pAND[0]], 'AND', sentence[pAND[1]:]
    else:
        return None

In [12]:
# Tests
assert split_sentence('(B or C) and (D or E) and (F or G)') == ('(B or C)', 'AND', '(D or E) and (F or G)')
assert split_sentence('(B and C) or (D and E) and (F or G)') == ('(B and C)', 'OR', '(D and E) and (F or G)')
assert split_sentence('(A and B and C and D)') is None
assert split_sentence('(A or B and C or D)') is None

assert split_sentence('A and B and C and D') == ('A', 'AND', 'B and C and D')
assert split_sentence('A is B similar-to C') is None

## Step 3: To remove the brackets

In [13]:
# This is highly verbose version.

def remove_brackets(sentence:str) -> str:
    sentence = sentence.strip()
    if sentence[0] != '(':
        print("** '{}' ==> Not removing brackets unless the first character is '('".format(sentence))
        return sentence

    if sentence[-1] != ')':
        print("** '{}' ==> Not removing brackets unless the last character is ')'".format(sentence))
        return sentence

    closing = 0
    count = 0
    for i, x in enumerate(sentence):
        if x == '(':
            count += 1
            if count == 1 and closing != 0:
                    print("** '{}' ==> Not removing brackets. You need to split the sentence first.".format(sentence))
                    return sentence
                    
        elif x == ')':
            count -= 1
            if count == 0:
                closing = i
        
        if count < 0:
            raise ValueError("Incorrect ')' detected at position {} of '{}'".format(i, sentence))

    return sentence[1:-1]  

In [14]:
remove_brackets(' (P and Q) or (B or C) ')

** '(P and Q) or (B or C)' ==> Not removing brackets. You need to split the sentence first.


'(P and Q) or (B or C)'

In [15]:
assert remove_brackets('((P and Q) or (B or (C and D)))')  == '(P and Q) or (B or (C and D))'
assert remove_brackets('((A and B) or (C and D))') == '(A and B) or (C and D)'

In [16]:
try:
    remove_brackets(' (A or B)) and (C or D)')    
except ValueError as e:
    print("ValueError: {}".format(e))

ValueError: Incorrect ')' detected at position 8 of '(A or B)) and (C or D)'


In [17]:
assert remove_brackets('A and (C or D)')  == 'A and (C or D)'

** 'A and (C or D)' ==> Not removing brackets unless the first character is '('


In [18]:
assert remove_brackets('(A and B) or C') == '(A and B) or C'

** '(A and B) or C' ==> Not removing brackets unless the last character is ')'


## Step 4: Putting all together

In [19]:
dd = {
    'A': 'Apollo', 
    'B': 'Boy', 
    'C': 'Cup',
}

# Note: There is no key of 'D' and 'E'.

In [20]:
# Turn off some verbose message.

def remove_brackets(sentence:str) -> str:
    sentence = sentence.strip()
    if sentence[0] != '(':
        return sentence

    if sentence[-1] != ')':
        return sentence

    closing = 0
    count = 0
    for i, x in enumerate(sentence):
        if x == '(':
            count += 1
            if count == 1 and closing != 0:
                    print("** '{}' ==> Not removing brackets. You need to split it first.".format(sentence))
                    return sentence
                    
        elif x == ')':
            count -= 1
            if count == 0:
                closing = i
        
        if count < 0:
            raise ValueError("Incorrect ')' detected at position {} of '{}'".format(i, sentence))

    return sentence[1:-1]  

In [21]:
def func2(sentence:str, dd:dict) -> str:
    sentence = remove_brackets(sentence)
    print("** Start  processing '{}'".format(sentence))
    s = split_sentence(sentence)

    if s and s[1] == 'OR':
        result = func2(s[0], dd) or func2(s[2], dd)
        print("** Start  processing '{}':  result='{}'".format(sentence, result))
        return result

    elif s and s[1] == 'AND':
        result = func2(s[0], dd) + func2(s[2], dd)
        print("** Finish processing '{}':  result='{}'".format(sentence, result))
        return result        

    else:
        if sentence in dd:
            result = dd[sentence]
            print("** Finish processing '{}':  result='{}'".format(sentence, result))
            return result                    
        else:
            print("** Finish processing '{}':  result=''".format(sentence))
            return ''        

In [22]:
result = func2('A and B and C', dd)
print("result:", result)

** Start  processing 'A and B and C'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'B and C'
** Start  processing 'B'
** Finish processing 'B':  result='Boy'
** Start  processing 'C'
** Finish processing 'C':  result='Cup'
** Finish processing 'B and C':  result='BoyCup'
** Finish processing 'A and B and C':  result='ApolloBoyCup'
result: ApolloBoyCup


In [23]:
result = func2('A and (B or C)', dd)
print("result:", result)

** Start  processing 'A and (B or C)'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'B or C'
** Start  processing 'B'
** Finish processing 'B':  result='Boy'
** Start  processing 'B or C':  result='Boy'
** Finish processing 'A and (B or C)':  result='ApolloBoy'
result: ApolloBoy


In [24]:
result = func2('(A or B) and C', dd)
print("result:", result)

** Start  processing '(A or B) and C'
** Start  processing 'A or B'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'A or B':  result='Apollo'
** Start  processing 'C'
** Finish processing 'C':  result='Cup'
** Finish processing '(A or B) and C':  result='ApolloCup'
result: ApolloCup


In [25]:
result = func2('A or (B and C)', dd)
print("result:", result)

** Start  processing 'A or (B and C)'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'A or (B and C)':  result='Apollo'
result: Apollo


In [26]:
result = func2('D or (A and B)', dd)
print("result:", result)

** Start  processing 'D or (A and B)'
** Start  processing 'D'
** Finish processing 'D':  result=''
** Start  processing 'A and B'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'B'
** Finish processing 'B':  result='Boy'
** Finish processing 'A and B':  result='ApolloBoy'
** Start  processing 'D or (A and B)':  result='ApolloBoy'
result: ApolloBoy


In [27]:
result = func2('(A or B) and (C or D) and (E or F)', dd)
print("result:", result)

** '(A or B) and (C or D) and (E or F)' ==> Not removing brackets. You need to split it first.
** Start  processing '(A or B) and (C or D) and (E or F)'
** Start  processing 'A or B'
** Start  processing 'A'
** Finish processing 'A':  result='Apollo'
** Start  processing 'A or B':  result='Apollo'
** '(C or D) and (E or F)' ==> Not removing brackets. You need to split it first.
** Start  processing '(C or D) and (E or F)'
** Start  processing 'C or D'
** Start  processing 'C'
** Finish processing 'C':  result='Cup'
** Start  processing 'C or D':  result='Cup'
** Start  processing 'E or F'
** Start  processing 'E'
** Finish processing 'E':  result=''
** Start  processing 'F'
** Finish processing 'F':  result=''
** Start  processing 'E or F':  result=''
** Finish processing '(C or D) and (E or F)':  result='Cup'
** Finish processing '(A or B) and (C or D) and (E or F)':  result='ApolloCup'
result: ApolloCup


## To test the given dictionary

In [28]:
sentence = 'A and B'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and B'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'B'
** Finish processing 'B':  result='World'
** Finish processing 'A and B':  result='HelloWorld'
result: HelloWorld


In [29]:
sentence = 'A and C'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and C'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'C'
** Finish processing 'C':  result='Buddy'
** Finish processing 'A and C':  result='HelloBuddy'
result: HelloBuddy


In [30]:
sentence = 'D or B'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'D or B'
** Start  processing 'D'
** Finish processing 'D':  result=''
** Start  processing 'B'
** Finish processing 'B':  result='World'
** Start  processing 'D or B':  result='World'
result: World


In [31]:
sentence = 'A or B'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A or B'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'A or B':  result='Hello'
result: Hello


In [32]:
sentence = 'A and B and C'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and B and C'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'B and C'
** Start  processing 'B'
** Finish processing 'B':  result='World'
** Start  processing 'C'
** Finish processing 'C':  result='Buddy'
** Finish processing 'B and C':  result='WorldBuddy'
** Finish processing 'A and B and C':  result='HelloWorldBuddy'
result: HelloWorldBuddy


In [33]:
sentence = 'A and (B or C)'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and (B or C)'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'B or C'
** Start  processing 'B'
** Finish processing 'B':  result='World'
** Start  processing 'B or C':  result='World'
** Finish processing 'A and (B or C)':  result='HelloWorld'
result: HelloWorld


In [34]:
sentence = 'A and (C or D)'
dd = {'A': 'Hello', 'B': 'World', 'C': 'Buddy'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and (C or D)'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing 'C or D'
** Start  processing 'C'
** Finish processing 'C':  result='Buddy'
** Start  processing 'C or D':  result='Buddy'
** Finish processing 'A and (C or D)':  result='HelloBuddy'
result: HelloBuddy


In [35]:
sentence = 'A and (B or C) and D'
dd = {'A': 'Hello', 'C': 'Buddy', 'D': 'Welcome'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and (B or C) and D'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing '(B or C) and D'
** Start  processing 'B or C'
** Start  processing 'B'
** Finish processing 'B':  result=''
** Start  processing 'C'
** Finish processing 'C':  result='Buddy'
** Start  processing 'B or C':  result='Buddy'
** Start  processing 'D'
** Finish processing 'D':  result='Welcome'
** Finish processing '(B or C) and D':  result='BuddyWelcome'
** Finish processing 'A and (B or C) and D':  result='HelloBuddyWelcome'
result: HelloBuddyWelcome


In [36]:
sentence = 'A and ((B or C) and D) and E'
dd = {'A': 'Hello', 'C': 'Buddy', 'D': 'Welcome', 'E': 'Elephant'}
result = func2(sentence, dd)
print("result:", result)

** Start  processing 'A and ((B or C) and D) and E'
** Start  processing 'A'
** Finish processing 'A':  result='Hello'
** Start  processing '((B or C) and D) and E'
** Start  processing '(B or C'
** Start  processing '(B'
** Finish processing '(B':  result=''
** Start  processing 'C'
** Finish processing 'C':  result='Buddy'
** Start  processing '(B or C':  result='Buddy'
** Start  processing 'D) and E'
** Start  processing 'D)'
** Finish processing 'D)':  result=''
** Start  processing 'E'
** Finish processing 'E':  result='Elephant'
** Finish processing 'D) and E':  result='Elephant'
** Finish processing '((B or C) and D) and E':  result='BuddyElephant'
** Finish processing 'A and ((B or C) and D) and E':  result='HelloBuddyElephant'
result: HelloBuddyElephant


## Extra tests